# [STARTER] Udaplay Project

## Part 02 - Agent

In this part of the project, you'll use your VectorDB to be part of your Agent as a tool.

You're building UdaPlay, an AI Research Agent for the video game industry. The agent will:
1. Answer questions using internal knowledge (RAG)
2. Search the web when needed
3. Maintain conversation state
4. Return structured outputs
5. Store useful information for future use

### Setup

In [1]:
# TODO: Import the necessary libs
# For example: 
import os

import json
from pydantic import BaseModel
from openai import OpenAI
from dotenv import load_dotenv
from tavily import TavilyClient



In [2]:
# TODO: Load environment variables
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")

### Tools

Build at least 3 tools:
- retrieve_game: To search the vector DB
- evaluate_retrieval: To assess the retrieval performance
- game_web_search: If no good, search the web


#### Retrieve Game Tool

In [3]:

import chromadb
from sentence_transformers import SentenceTransformer

# TODO: Create retrieve_game tool
# It should use chroma client and collection you created

# Tool Docstring:
#    Semantic search: Finds most results in the vector DB
#    args:
#    - query: a question about game industry. 
#
#    You'll receive results as list. Each element contains:
#    - Platform: like Game Boy, Playstation 5, Xbox 360...)
#    - Name: Name of the Game
#    - YearOfRelease: Year when that game was released for that platform
#    - Description: Additional details about the game

# Use a small, fast model for local embeddings
embedder = SentenceTransformer('all-MiniLM-L6-v2')
chroma_client = chromadb.PersistentClient(path="chroma_db_jupiter")

def embed(texts: list[str]):
    """Return list of vectors (lists of floats)."""
    vectors = embedder.encode(texts, convert_to_numpy=True)
    return vectors.tolist()

def retrieve_game(query: str, n_results: int = 3):
    query_vector = embed([query])[0]
    collection = chroma_client.get_collection("games_collection_new")
    results = collection.query(
        query_embeddings=[query_vector],
        n_results=n_results,
        include=['documents','metadatas']
    )
    metadatas = results.get('metadatas', [[]])[0]
    documents = results.get('documents', [[]])[0]
    out = []
    for meta, doc in zip(metadatas, documents):
        out.append({
            'Platform': meta.get('Platform'),
            'Name': meta.get('Name'),
            'YearOfRelease': meta.get('YearOfRelease'),
            'Description': meta.get('Description'),
            'document': doc
        })
    return out

C:\Projects\building-agents\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


#### Evaluate Retrieval Tool

In [4]:
# TODO: Create evaluate_retrieval tool
# You might use an LLM as judge in this tool to evaluate the performance
# You need to prompt that LLM with something like:
# "Your task is to evaluate if the documents are enough to respond the query. "
# "Give a detailed explanation, so it's possible to take an action to accept it or not."
# Use EvaluationReport to parse the result
# Tool Docstring:
#    Based on the user's question and on the list of retrieved documents, 
#    it will analyze the usability of the documents to respond to that question. 
#    args: 
#    - question: original question from user
#    - retrieved_docs: retrieved documents most similar to the user query in the Vector Database
#    The result includes:
#    - useful: whether the documents are useful to answer the question
#    - description: description about the evaluation result

client = OpenAI(api_key=os.getenv('OPENAI_API_KEY'), base_url=os.getenv('OPENAI_BASE_URL'))

class EvaluationReport(BaseModel):
    useful: bool
    description: str

def evaluate_retrieval(question: str, retrieved_docs: list[dict]):
    prompt = f"""Your task is to decide whether the retrieved documents are sufficient to answer the question.

Question:
{question}

Retrieved Documents:
{json.dumps(retrieved_docs, indent=2)}

Explain whether the documents contain enough information. Then return valid JSON exactly like:
{{"useful": true, "description": "..."}}
"""

    resp = client.chat.completions.create(
        model=os.getenv('EVAL_MODEL', 'gpt-3.5-turbo'),
        messages=[{'role': 'user', 'content': prompt}],
        temperature=0
    )
    content = resp.choices[0].message.content
    # parse JSON
    try:
        parsed = json.loads(content)
        return EvaluationReport.model_validate(parsed)
    except Exception:
        # fallback: treat as not useful and include raw output
        return EvaluationReport(useful=False, description='Could not parse judge output: ' + content)

#### Game Web Search Tool

In [5]:
# TODO: Create game_web_search tool
# Please use Tavily client to search the web
# Tool Docstring:
#    Semantic search: Finds most results in the vector DB
#    args:
#    - question: a question about game industry. load_dotenv()

tavily = TavilyClient(api_key=os.getenv('TAVILY_API_KEY'))

def game_web_search(question: str, max_results: int = 3):
    try:
        resp = tavily.search(query=question, include_answer=True, max_results=max_results)
        return {'answer': resp.get('answer'), 'results': resp.get('results')}
    except Exception as e:
        return {'answer': None, 'results': [], 'error': str(e)}

### Agent

In [10]:
# TODO: Create your Agent abstraction using StateMachine
# Equip with an appropriate model
# Craft a good set of instructions 
# Plug all Tools you developed
client = OpenAI(api_key=os.getenv('OPENAI_API_KEY'), base_url=os.getenv('OPENAI_BASE_URL'))

def run_agent(question: str):
    print('🔎 Retrieving from local DB...')
    local_results = retrieve_game(question)

    print('🧠 Evaluating retrieval quality...')
    evaluation = evaluate_retrieval(question, local_results)

    if evaluation.useful:
        return {
            'source': 'local',
            'data': local_results,
            'explanation': evaluation.description
        }

    print('🌐 Local data insufficient → using web search...')
    web = game_web_search(question)

    # Ask LLM to synthesize the web output + local results
    prompt = f"""The local database did not contain enough information.

Question: {question}

Local results: {json.dumps(local_results, indent=2)}

Web search results: {json.dumps(web, indent=2)}

Provide a concise, accurate answer to the user."""

    resp = client.chat.completions.create(
        model=os.getenv('ANSWER_MODEL', 'gpt-5'),
        messages=[{'role': 'user', 'content': prompt}],
        temperature=0.7
    )
    final = resp.choices[0].message.content
    return {'source': 'web', 'data': final, 'explanation': evaluation.description}

In [12]:
# TODO: Invoke your agent
# - When Pokémon Gold and Silver was released?
# - Which one was the first 3D platformer Mario game?
# - Was Mortal Kombat X realeased for Playstation 5?
if __name__ == '__main__':
    q = input('Ask a question about video games (or type exit): ').strip()
    out = run_agent(q)
    print('\n=== RESULT ===')
    print('Source:', out['source'])
    print('Explanation:', out['explanation'])
    print('Data:', out['data'])
    print('\n')

🔎 Retrieving from local DB...
🧠 Evaluating retrieval quality...

=== RESULT ===
Source: local
Explanation: The retrieved documents contain the necessary information to answer the question. The year of release of Super Mario 64 is 1996, as stated in the first document.
Data: [{'Platform': 'Nintendo 64', 'Name': 'Super Mario 64', 'YearOfRelease': 1996, 'Description': "A groundbreaking 3D platformer that set new standards for the genre, featuring Mario's quest to rescue Princess Peach.", 'document': "[Nintendo 64] Super Mario 64 (1996) - A groundbreaking 3D platformer that set new standards for the genre, featuring Mario's quest to rescue Princess Peach."}, {'Platform': 'Super Nintendo Entertainment System (SNES)', 'Name': 'Super Mario World', 'YearOfRelease': 1990, 'Description': 'A classic platformer where Mario embarks on a quest to save Princess Toadstool and Dinosaur Land from Bowser.', 'document': '[Super Nintendo Entertainment System (SNES)] Super Mario World (1990) - A classic pla

### (Optional) Advanced

In [ ]:
# TODO: Update your agent with long-term memory
# TODO: Convert the agent to be a state machine, with the tools being pre-defined nodes